In [ ]:
pip install keras-tuner --upgrade

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, MobileNetV3Small
from tensorflow.keras import layers, models
from keras_tuner import HyperModel
from kerastuner.tuners import RandomSearch

# Define the hypermodel
class WasteClassificationHyperModel(HyperModel):
    def __init__(self, dataset_directory, num_classes):
        self.dataset_directory = dataset_directory
        self.num_classes = num_classes
    
    def build(self, hp):
        # Choose the model
        model_type = hp.Choice('model_type', ['InceptionV3', 'MobileNetV3'])
        if model_type == 'InceptionV3':
            base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        else:
            base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        
        # Freeze the base model
        base_model.trainable = False
        
        # Define the model
        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
            layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        # Compile the model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# Define the data generator with augmentation
def create_data_generator(hp, dataset_directory):
    augmentation_technique = hp.Choice('augmentation_technique', ['none', 'flip', 'rotation', 'zoom'])
    
    if augmentation_technique == 'flip':
        datagen = ImageDataGenerator(horizontal_flip=True)
    elif augmentation_technique == 'rotation':
        datagen = ImageDataGenerator(rotation_range=20)
    elif augmentation_technique == 'zoom':
        datagen = ImageDataGenerator(zoom_range=0.2)
    else:
        datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        dataset_directory,
        target_size=(224, 224),
        batch_size=hp.Int('batch_size', min_value=16, max_value=64, step=16),
        class_mode='categorical'
    )
    
    return generator

# Define the dataset directory and number of classes
dataset_directory = '/path/to/dataset'
num_classes = 5  # Set the number of classes

# Create the hypermodel
hypermodel = WasteClassificationHyperModel(dataset_directory, num_classes)

# Define the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuning_results',
    project_name='waste_classification'
)

# Define the search space and perform tuning
tuner.search(
    create_data_generator(tuner.oracle.hyperparameters, dataset_directory),
    epochs=10,
    validation_data=create_data_generator(tuner.oracle.hyperparameters, dataset_directory)
)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the summary of the best model
best_model.summary()


2024-08-16 14:44:53.087786: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 14:44:53.097204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 14:44:53.108300: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 14:44:53.111585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 14:44:53.119899: I tensorflow/core/platform/cpu_feature_guar

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/dataset'